In [2]:
# !pip install optuna

  Using cached https://files.pythonhosted.org/packages/85/ee/2688cce5ced0597e12832d1ec4f4383a468f6bddff768eeaa3b5bf4f6500/optuna-1.3.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/60/1e/cabc75a189de0fbb2841d0975243e59bde8b7822bacbb95008ac6fe9ad47/alembic-1.4.2.tar.gz
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/bin/python /opt/anaconda3/lib/python3.7/site-packages/pip install --ignore-installed --no-user --prefix /tmp/pip-build-env-rbcvr38r/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- 'setuptools>=40.8.0' wheel
       cwd: None
  Complete output (14 lines):
  Traceback (most recent call last):
    File "/opt/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/anaconda3/lib/python3.

In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import optuna

DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


def define_model(trial):
    # We optimize the number of layers, hidden untis and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_uniform("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load MNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.MNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the MNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)

            # Zeroing out gradient buffers.
            optimizer.zero_grad()
            # Performing a forward pass.
            output = model(data)
            # Computing negative Log Likelihood loss.
            loss = F.nll_loss(output, target)
            # Performing a backward pass.
            loss.backward()
            # Updating the weights.
            optimizer.step()

    # Validation of the model.
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(valid_loader):
            # Limiting validation data.
            if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                break
            data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability.
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / N_VALID_EXAMPLES
    return accuracy

In [2]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Extracting /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw/train-images-idx3-ubyte.gz to /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw


Extracting /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw/train-labels-idx1-ubyte.gz to /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw



Extracting /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw


Extracting /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/jupyter/cassava_disease_classification/salomon_exp/MNIST/raw
Processing...
Done!





[I 2020-05-01 01:09:31,575] Finished trial#0 with value: 0.81328125 with parameters: {'n_layers': 1, 'n_units_l0': 10, 'dropout_l0': 0.4954869800966186, 'optimizer': 'SGD', 'lr': 0.05886382632216787}. Best is trial#0 with value: 0.81328125.
[I 2020-05-01 01:09:37,441] Finished trial#1 with value: 0.11015625 with parameters: {'n_layers': 3, 'n_units_l0': 73, 'dropout_l0': 0.3843838583578286, 'n_units_l1': 63, 'dropout_l1': 0.23683135777045417, 'n_units_l2': 41, 'dropout_l2': 0.4954059008268575, 'optimizer': 'SGD', 'lr': 0.0004818731785282965}. Best is trial#0 with value: 0.81328125.
[I 2020-05-01 01:09:43,852] Finished trial#2 with value: 0.1078125 with parameters: {'n_layers': 2, 'n_units_l0': 10, 'dropout_l0': 0.3249955426549393, 'n_units_l1': 88, 'dropout_l1': 0.34705682711740027, 'optimizer': 'SGD', 'lr': 5.5280630823700153e-05}. Best is trial#0 with value: 0.81328125.
[I 2020-05-01 01:09:50,025] Finished trial#3 with value: 0.7109375 with parameters: {'n_layers': 1, 'n_units_l0': 2

Number of finished trials:  100
Best trial:
  Value:  0.9546875
  Params: 
    n_layers: 1
    n_units_l0: 117
    dropout_l0: 0.33666940899323733
    optimizer: RMSprop
    lr: 0.0028539230608814444
